In [1]:
!pip install transformers tqdm boto3 requests regex -q

     |████████████████████████████████| 573kB 2.8MB/s 
     |████████████████████████████████| 1.0MB 64.3MB/s 
     |████████████████████████████████| 3.7MB 75.1MB/s 
     |████████████████████████████████| 890kB 62.2MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
cd gdrive/My\ Drive/Colab Notebooks

/content/gdrive/My Drive/Colab Notebooks


In [0]:
import pandas as pd

In [5]:
df_train = pd.read_excel("stock_data/stockdata_withdo_new_fiftycompany_train.xlsx")
df_test = pd.read_excel("stock_data/stockdata_withdo_new_fiftycompany_test.xlsx")
df_dev = pd.read_excel("stock_data/stockdata_withdo_new_fiftycompany_dev.xlsx")

ratio = len(df_test) / len(df_train)
print("測試集樣本數 / 訓練集樣本數 = {:.2f} 倍".format(ratio))


測試集樣本數 / 訓練集樣本數 = 0.12 倍


In [0]:
import pandas as pd
from pandas import DataFrame 
l1 = []
l2 = []

for i, row in df_train.iterrows():
    if row['start_price'] and row['last_price']:
        content = row['content'][:100]
        l1.extend([content])
        l2.extend([row['labeling']])

df_train = DataFrame({'content': l1, 'labeling': l2})

l1 = []
l2 = []

for i, row in df_test.iterrows():
    if row['start_price'] and row['last_price']:
        content = row['content'][:100]
        l1.extend([content])
        l2.extend([row['labeling']])
    
df_test = DataFrame({'content': l1, 'labeling': l2})

l1 = []
l2 = []

for i, row in df_dev.iterrows():
    if row['start_price'] and row['last_price']:
        content = row['content'][:100]
        l1.extend([content])
        l2.extend([row['labeling']])
    
df_dev = DataFrame({'content': l1, 'labeling': l2})

In [0]:
# 去除不必要欄位

## (測試用)
# SAMPLE_FRAC = 0.001
# df_train = df_train.sample(frac=SAMPLE_FRAC, random_state=9527)

df_train = df_train.reset_index()
df_train = df_train.loc[:, ['content', 'labeling']]
df_train.columns = ['content', 'labeling']
df_train.to_csv("train.tsv", sep="\t", index=False)

df_test = df_test.loc[:, ['content', 'labeling']]
df_test.columns = ['content', 'labeling']
df_test.to_csv("test.tsv", sep="\t", index=False)

df_dev = df_dev.loc[:, ['content', 'labeling']]
df_dev.columns = ['content', 'labeling']
df_dev.to_csv("dev.tsv", sep="\t", index=False)

In [8]:
print(len(df_train))

71697


In [9]:
"""
實作一個可以用來讀取訓練 / 測試集的 Dataset，這是你需要徹底了解的部分。
此 Dataset 每次將 tsv 裡的一筆成對句子轉換成 BERT 相容的格式，並回傳 3 個 tensors：
- tokens_tensor：兩個句子合併後的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用來識別兩個句子界限的 binary tensor
- label_tensor：將分類標籤轉換成類別索引的 tensor, 如果是測試集則回傳 None
"""

from torch.utils.data import Dataset
from transformers import BertTokenizer
import torch

PRETRAINED_MODEL_NAME = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
    
class StockDataset(Dataset):
    # 讀取前處理後的 tsv 檔並初始化一些參數
    def __init__(self, mode, tokenizer):
        assert mode in ["train", "test", "dev"]  # 一般訓練你會需要 dev set
        self.mode = mode
        # 大數據你會需要用 iterator=True
        self.df = pd.read_csv(mode + ".tsv", sep="\t").fillna("")
        self.len = len(self.df)
        self.tokenizer = tokenizer  # 我們將使用 BERT tokenizer
    
    # 定義回傳一筆訓練 / 測試數據的函式
    def __getitem__(self, idx):
        if self.mode == "test":
            content, label = self.df.iloc[idx, :2].values
            label_id = label
            label_tensor = torch.tensor(label_id)
        else:
            content, label = self.df.iloc[idx, :2].values
            # 將 label 文字也轉換成索引方便轉換成 tensor
            label_id = label
            label_tensor = torch.tensor(label_id)
            
        # 建立第一個句子的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = ["[CLS]"]
        tokens_a = self.tokenizer.tokenize(content)
        word_pieces += tokens_a + ["[SEP]"]
        len_a = len(word_pieces)
                
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        # 將第一句包含 [SEP] 的 token 位置設為 0，其他為 1 表示第二句
        segments_tensor = torch.tensor([0] * len_a , 
                                        dtype=torch.long)
        
        return (tokens_tensor, segments_tensor, label_tensor)
    
    def __len__(self):
        return self.len
    
    
# 初始化一個專門讀取訓練樣本的 Dataset，使用中文 BERT 斷詞
trainset = StockDataset("train", tokenizer=tokenizer)

devset = StockDataset("dev", tokenizer=tokenizer)

In [10]:
# 選擇第一個樣本
sample_idx = 1

# 將原始文本拿出做比較
content, labeling = trainset.df.iloc[sample_idx].values

# 利用剛剛建立的 Dataset 取出轉換後的 id tensors
tokens_tensor, segments_tensor, label_tensor = trainset[sample_idx]

# 將 tokens_tensor 還原成文本
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = "".join(tokens)

# 渲染前後差異，毫無反應就是個 print。可以直接看輸出結果
print(f"""[原始文本]
句子 1：{content}
分類  ：{labeling}

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：{tokens_tensor}

segments_tensor：{segments_tensor}

label_tensor   ：{label_tensor}

--------------------

[還原 tokens_tensors]
{combined_text}
""")


[原始文本]
句子 1： 澳洲 央行 公佈 現金 利率 目標 蘋果 WWDC 大會 重要 新聞 大立光 5月 營收 3757億 元 為 今年 次 高 i8 鏡頭 7月 開始 出貨 more TPK 5月 營收 7114億 月
分類  ：0

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：tensor([  101,  4078,  3828,  1925,  6121,  1062,   854,  4412,  7032,  1164,
         4372,  4680,  3560,  5981,  3362,   100,  1920,  3298,  7028,  6206,
         3173,  5472,  1920,  4989,  1045,   126,  3299,  4245,  3119, 11256,
         8161,  1023,  1039,  4158,   791,  2399,  3613,  7770,   151,  8156,
         7128,  7531,   128,  3299,  7274,  1993,  1139,  6515,  8384,   100,
          126,  3299,  4245,  3119, 12565,  8159,  1023,  3299,   102])

segments_tensor：tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

label_tensor   ：0

--------------------

[還原 tokens_tensors]
[CLS]澳洲央行公佈現金利率目標蘋果[UNK]大會重要新聞大立光5月營收375##7億元為今年次高i##8鏡頭7月開始出貨more[UNK]5月營收7

In [0]:
"""
實作可以一次回傳一個 mini-batch 的 DataLoader
這個 DataLoader 吃我們上面定義的 `FakeNewsDataset`，
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 這個函式的輸入 `samples` 是一個 list，裡頭的每個 element 都是
# 剛剛定義的 `FakeNewsDataset` 回傳的一個樣本，每個樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 它會對前兩個 tensors 作 zero padding，並產生前面說明過的 masks_tensors
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 訓練集有 labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列長度
    tokens_tensors = pad_sequence(tokens_tensors, 
                                  batch_first=True)
    segments_tensors = pad_sequence(segments_tensors, 
                                    batch_first=True)
    
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape, 
                                dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(
        tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids


# 初始化一個每次回傳 64 個訓練樣本的 DataLoader
# 利用 `collate_fn` 將 list of samples 合併成一個 mini-batch 是關鍵
BATCH_SIZE = 16
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, 
                         collate_fn=create_mini_batch)

devloader = DataLoader(devset, batch_size=BATCH_SIZE, 
                        collate_fn=create_mini_batch)

In [12]:
data = next(iter(trainloader))

tokens_tensors, segments_tensors, \
    masks_tensors, label_ids = data

print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
segments_tensors.shape = {segments_tensors.shape}
{segments_tensors}
------------------------
masks_tensors.shape    = {masks_tensors.shape}
{masks_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")


tokens_tensors.shape   = torch.Size([16, 65]) 
tensor([[ 101,  464, 5836,  ..., 7065, 6121,  102],
        [ 101, 4078, 3828,  ...,    0,    0,    0],
        [ 101,  126, 7442,  ...,  102,    0,    0],
        ...,
        [ 101,  127, 1168,  ...,    0,    0,    0],
        [ 101, 1378, 4948,  ...,    0,    0,    0],
        [ 101, 1912, 6536,  ...,    0,    0,    0]])
------------------------
segments_tensors.shape = torch.Size([16, 65])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
------------------------
masks_tensors.shape    = torch.Size([16, 65])
tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
------------------------
label_ids.shape        

In [13]:
from transformers import BertForSequenceClassification

PRETRAINED_MODEL_NAME = "bert-base-chinese"
NUM_LABELS = 3

model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)

# model = torch.load('Bert_model/Bert50_300_25.pkl')

#clear_output()

# high-level 顯示此模型裡的 modules
print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))



name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=3, bias=True)


In [14]:
model.config

BertConfig {
  "_num_labels": 3,
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "directionality": "bidi",
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "min_length": 0,
  "model_type": "bert",
  "no_repeat_ngram_size": 0,
  "num_attention_heads": 12,
  "num_beams": 1,
  "num_hidden_layers": 12,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden

In [15]:
"""
定義一個可以針對特定 DataLoader 取得模型預測結果以及分類準確度的函式

GPU 跑會有 cuda runtime error 的問題，所以先使用 CPU 跑不會有狀況!

"""

def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
    tp = 0
    tn = 0
    fp = 0
    fn = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()

                tp += (labels * pred).sum().to(torch.float32)
                tn += ((1 - labels) * (1 - pred)).sum().to(torch.float32)
                fp += ((1 - labels) * pred).sum().to(torch.float32)
                fn += (labels * (1 - pred)).sum().to(torch.float32)
                                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total

        #epsilon = 1e-7
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        fscore = (2 * precision * recall) / (precision + recall)
        return predictions, acc, precision, recall, fscore    
    return predictions
  

# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
# _, acc, precision, recall, fscore = get_predictions(model, trainloader, compute_acc=True)
# print("classification acc:", acc)


device: cuda:0


In [16]:
def get_learnable_params(module):
    return [p for p in module.parameters() if p.requires_grad]
     
model_params = get_learnable_params(model)
clf_params = get_learnable_params(model.classifier)

print(f"""
整個分類模型的參數量：{sum(p.numel() for p in model_params)}
線性分類器的參數量：{sum(p.numel() for p in clf_params)}
""")


整個分類模型的參數量：102269955
線性分類器的參數量：2307



In [0]:
%%time

import time

# 訓練模式
model.train()

# 使用 Adam Optim 更新整個分類模型的參數
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

EPOCHS = 100 # 預設為 6
for epoch in range(EPOCHS):
    
    tStart=time.time()
    running_loss = 0.0
    for data in trainloader:
        
        tokens_tensors, segments_tensors, \
        masks_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)

        loss = outputs[0]
        # backward
        loss.backward()
        optimizer.step()

        # 紀錄當前 batch loss
        running_loss += loss.item()
 
    # 計算分類準確率
    if (epoch+1) % 5 == 0 and epoch > 0 :
        # 計算分類準確率
        _, acc, precision, recall, fscore = get_predictions(model, devloader, compute_acc=True)

        print('[epoch %d] loss: %.3f, acc: %.3f, precision: %.3f, recall: %.3f, fscore: %.3f' % 
              (epoch + 1, running_loss, acc, precision, recall, fscore))
    
        tEnd=time.time()
        print("\n It cost %f sec"%(tEnd-tStart))
torch.save(model, 'Bert_new50.pkl')


[epoch 5] loss: 2065.128, acc: 0.606, precision: 0.619, recall: 0.790, fscore: 0.694

 It cost 646.679185 sec


In [0]:
## 儲存model的參數
# torch.save(model.state_dict(), 'Bert_withdo50.pkl')

In [0]:
%%time

# 建立測試集
testset = StockDataset("test", tokenizer=tokenizer)
testloader = DataLoader(testset, batch_size=32, 
                        collate_fn=create_mini_batch)

_, acc, precision, recall, fscore = get_predictions(model, testloader, compute_acc=True)

print('acc: %.3f, precision: %.3f, recall: %.3f, fscore: %.3f' % 
      (acc, precision, recall, fscore))